In [160]:
from functools import partial, partialmethod, wraps
import json
from pathlib import Path

class Meta(type):
    def __new__(
        cls, clsname, bases, dct, 
        cache_root=None, 
        cache_type_name=None,
        cache_key='id',
        cache_dir_key='_dir',
        debug=None,
    ):
        def print(*args, **kwargs):
            if debug:
                debug(*args, **kwargs)
        
        print(f'Meta.__new__({cls}, {clsname}, {bases}, {dct})')
        methods = dct.copy()
        fields = []

        if not cache_root:
            cache_root = Path.cwd() / '.objs'
        cache_root = Path(cache_root)

        if not cache_type_name:
            cache_type_name = clsname
            
        class_dir = cache_root / cache_type_name
        class_dir.mkdir(parents=True, exist_ok=True)
        print(f'Class cache dir: {class_dir}')

        for name, fn in dct.items():
            print(f'Checking: {name}: {fn}')
            if isinstance(fn, property):
                print(f'property: {name} -> {fn}')
                fields.append(name)
                field_name = f'_{name}'
                
                def wrapped_fn(self, name, field_name, fn, *args, **kwargs):
                    print(f'wrapped_fn: {name} ({field_name}): {fn}')
                    if not hasattr(self, field_name):
                        
                        # Path to cache this instance's fields to
                        cache_dir = getattr(self, cache_dir_key)

                        # Path to cache this specific field to (within the instance's dir)
                        path = cache_dir / name
                        if path.exists():
                            with path.open('r') as f:
                                val = json.load(f)
                            print(f'Loaded attr from cache: {field_name}={val}')
                        else:
                            print(f'Computing: {name}')
                            assert isinstance(fn, property), fn

                            val = fn.fget(self, *args, **kwargs)
                            print(f'Computed: {name}={val}')

                            cache_dir.mkdir(parents=True, exist_ok=True)

                            print(f'Saving {name} to {path}')
                            with path.open('w') as f:
                                json.dump(val, f)

                        # set loaded value on instance
                        setattr(self, field_name, val)
                    else:
                        print(f'Lookup: {name}={getattr(self, field_name)}')

                    return getattr(self, field_name)

                applied = partial(wrapped_fn, name=name, field_name=field_name, fn=fn)
                prop = property(applied)
                print(f'Setting method: {name}: {prop}')
                methods[name] = prop
            else:
                print(f'Skipping: {name}')
                    
        print(f'Fields: {fields}')
        orig_init = None
        if '__init__' in methods:
            orig_init = methods['__init__']

        def __init__(self, *args, **_kwargs):
            kwargs = _kwargs.copy()
            print(f'__init__(self, {args}, {kwargs})')

            if cache_key in kwargs:
                cache_id = kwargs.pop(cache_key)
            elif args:
                cache_id = args[0]
                args = args[1:]
            else:
                raise Exception(f"Couldn't find {cache_key} field in kwargs or as first element of *args ")

            print(f'Setting cache id {cache_key}={cache_id}')    
            setattr(self, cache_key, cache_id)
            
            cache_dir = class_dir / str(cache_id)
            setattr(self, cache_dir_key, cache_dir)
            print(f'Set {cache_dir_key}={cache_dir} for id {cache_id}')
                    
            if orig_init:
                orig_init(self, *args, **kwargs)
            
            print(f'Done with injected __init__')
        
        methods['__init__'] = __init__
        
        def __str__(self):
            strs = [ f'{cache_key}={getattr(self, cache_key)}' ]
            for field in fields:
                k = f'_{field}'
                if hasattr(self, k):
                    v = getattr(self, k)
                    strs.append(f'{field}={v}')
            return '%s(%s)' % (clsname, ', '.join(strs))
                
        if '__str__' not in methods:
            methods['__str__'] = __str__

        if '__repr__' not in methods:
            methods['__repr__'] = __str__

        new = super(Meta, cls).__new__(cls, clsname, bases, methods)
        print(f'returning {new}: {methods}')

        return new

In [167]:
class Foo(metaclass=Meta):
    @property
    def a(self):
        return f'{self.id}-a'

    @property
    def b(self):
        return self.id * 2

In [168]:
f = Foo(111); f

Foo(id=111)

In [170]:
f.a, str(f)

('111-a', 'Foo(id=111, a=111-a)')

In [171]:
f.b, str(f)

(222, 'Foo(id=111, a=111-a, b=222)')

In [173]:
f2 = Foo(222); f2

Foo(id=222)

In [174]:
f2.a, str(f2)

('222-a', 'Foo(id=222, a=222-a)')

In [175]:
f2.b, str(f2)

(444, 'Foo(id=222, a=222-a, b=444)')